# Source Detection

This notebook is an attempt to split out the source detection and measurement algorithms from `processCCD` and apply them to the search for low surface brightness galaxies. This builds off Robert Lupton's [Greco LSB.ipynb](https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Greco%20LSB.ipynb) with some source detection and measurement details from [Tune Detection.ipynb](https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Tune%20Detection.ipynb) and [Kron.ipynb](https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Kron.ipynb).

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import IFrame, display, Markdown

In [ ]:
from astropy.visualization import ZScaleInterval
zscale = ZScaleInterval()

from matplotlib.patches import Rectangle

In [ ]:
import lsst.daf.persistence as dafPersist
import lsst.daf.base as dafBase

import lsst.afw.detection as afwDetect
import lsst.afw.image as afwImage
import lsst.afw.math as afwMath
import lsst.afw.geom as afwGeom

import lsst.daf.persistence        as dafPersist
import lsst.afw.display            as afwDisplay
import lsst.afw.table              as afwTable

In [ ]:
datadir = "/project/shared/data/Twinkles_subset/output_data_v2"
butler = dafPersist.Butler(datadir)

In [ ]:
def smooth_gauss (masked_image, sigma, nsigma=1.0):
    width = (int(sigma*nsigma + 0.5) // 2)*2 + 1
    gauss_func = afwMath.GaussianFunction1D(sigma)
    gauss_kern = afwMath.SeparableKernel(width, width, gauss_func, gauss_func)
    convolved_image = masked_image.Factory(masked_image.getBBox())
    afwMath.convolve(convolved_image, masked_image, gauss_kern, 
                     afwMath.ConvolutionControl())
    return convolved_image

def display_image(image, figsize=(15,5)):
    fig, ax = plt.subplots(figsize=figsize)
    vmin, vmax = zscale.get_limits(image)
    ax.imshow(image, vmin=vmin, vmax=vmax, cmap='gray_r', 
              aspect='equal', origin='lower')
    ax.axis('off')
    return ax

In [ ]:
print(butler.queryMetadata('calexp', ['visit','raft','sensor']))
dataId = {'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}
calexp = butler.get('calexp', **dataId)

In [ ]:
display_image(calexp.getMaskedImage().getImage().getArray(), figsize=(8,8));
xmin,ymin = 3600,2150
width,height=300,300
rect = Rectangle((xmin,ymin),width,height,facecolor='none',edgecolor='r',lw=2)
plt.gca().add_artist(rect)

In [ ]:
# Zoom to a small region of the image
bbox = afwGeom.Box2I()
bbox.include(afwGeom.Point2I(xmin, ymin))
bbox.include(afwGeom.Point2I(xmin+width, ymin+height))
exp_cutout = calexp.Factory(calexp, bbox, afwImage.LOCAL)

mi = exp_cutout.getMaskedImage()
#mask = mi.getMask()
## Careful, this is clearing the mask plane of the parent calexp
#mask.removeAndClearMaskPlane('CR', True)

display_image(mi.getImage().getArray());

In [ ]:
mi = calexp.getMaskedImage()
mask = mi.getMask()
mask.getMaskPlaneDict()

# Source Detection and Measurement

We now want to run the LSST source detection and measurement tasks.

In [ ]:
# Importing the tasks
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.pipe.tasks.calibrate         import CalibrateTask
from lsst.meas.algorithms.detection    import SourceDetectionTask
from lsst.meas.deblender               import SourceDeblendTask
from lsst.meas.base                    import SingleFrameMeasurementTask

In [ ]:
# Create the Tasks
schema = afwTable.SourceTable.makeMinimalSchema()
algMetadata = dafBase.PropertyList()

config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask = CharacterizeImageTask(None, config=config)

config = SourceDetectionTask.ConfigClass()

# SourceDetectionTask.thresholdType: 
#print(SourceDetectionTask.ConfigClass.thresholdType.__doc__)
if True:
    config.thresholdValue = 30       # detection threshold in units of thresholdType
    config.thresholdType = "stdev"   # units for thresholdValue
if False:                    
    config.doTempLocalBackground = True  # Use local-background during detection step

sourceDetectionTask =   SourceDetectionTask(schema=schema, config=config)
sourceDeblendTask   =   SourceDeblendTask(schema=schema)

config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config,
                                                   algMetadata=algMetadata)

In [ ]:
help(sourceMeasurementTask.run)

In [ ]:
tab = afwTable.SourceTable.make(schema)

# Image characterization
result = charImageTask.characterize(calexp)

# Source detection
result = sourceDetectionTask.run(tab, calexp)
sources = result.sources
# Source deblending
sourceDeblendTask.run(calexp, sources)
# Source measurement
sourceMeasurementTask.run(measCat=sources, exposure=calexp)

if False:
    sources.writeFits("outputTable.fits")
    exposure.writeFits("example1-out.fits")
    
    

In [ ]:
# ADW: Why am I copying the sources?
sources = sources.copy(True)

good = np.logical_and.reduce([sources.get('base_PixelFlags_flag_saturatedCenter') == 0,
                              sources.get("deblend_nChild") == 0,
                              ])

In [ ]:
# Investigate the output source catalog
sources.asAstropy()

In [ ]:
# Setup the Firefly display
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'

# Note that the IFrame needs to be the last thing in the cell
ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
IFrame(ff,800,600)

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.getDisplay(frame=1, 
                                    name=my_channel)

In [ ]:
#Display the cutout with afw display
afw_display.mtv(exp_cutout)

with afw_display.Buffering():
    for s in sources:
        afw_display.dot('+', *s.getCentroid(), ctype=afwDisplay.RED)
        afw_display.dot('o', s.getX(), s.getY(), size=20, ctype='orange')   